In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install necessary packages
!pip install -q transformers datasets scikit-learn pandas numpy matplotlib seaborn tqdm

# Import libraries
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import time
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import warnings
warnings.filterwarnings('ignore')

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Mounted at /content/drive
Using device: cuda


In [2]:
# Define the path to your data
data_path = '/content/drive/MyDrive/complaints.csv'  # Adjust this path as needed

# Load the data
df = pd.read_csv(data_path)

# Display basic information
print("Dataset shape:", df.shape)
print("\nColumn names:", df.columns.tolist())
print("\nFirst few rows:")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/complaints.csv'

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# Distribution of complaints by product
plt.figure(figsize=(12, 6))
sns.countplot(y='Product', data=df, order=df['Product'].value_counts().index)
plt.title('Distribution of Complaints by Product')
plt.tight_layout()
plt.show()

# Map products to our target categories
product_to_category = {
    'Credit reporting, credit repair services, or other personal consumer reports': 0,
    'Credit reporting': 0,
    'Credit card': 0,
    'Prepaid card': 0,
    'Payday loan': 0,
    'Money transfer': 0,
    'Virtual currency': 0,

    'Debt collection': 1,

    'Consumer Loan': 2,
    'Vehicle loan or lease': 2,
    'Student loan': 2,

    'Mortgage': 3,
    'Payday loan, title loan, or personal loan': 3
}

# Create the target column
df['category'] = df['Product'].map(product_to_category)

# Filter out rows with no category
df = df.dropna(subset=['category'])
df['category'] = df['category'].astype(int)

# Check distribution of our target categories
plt.figure(figsize=(10, 5))
sns.countplot(x='category', data=df)
plt.title('Distribution of Target Categories')
plt.xticks([0, 1, 2, 3],
           ['Credit reporting', 'Debt collection', 'Consumer Loan', 'Mortgage'])
plt.tight_layout()
plt.show()

# Check the length of complaint narratives
df['text_length'] = df['Consumer complaint narrative'].str.len()
plt.figure(figsize=(10, 5))
sns.histplot(df['text_length'], bins=50, kde=True)
plt.title('Distribution of Complaint Narrative Length')
plt.xlabel('Number of Characters')
plt.ylabel('Frequency')
plt.xlim(0, 5000)  # Limit x-axis for better visualization
plt.tight_layout()
plt.show()

In [ ]:
# Define text preprocessing function
def preprocess_text(text):
    if pd.isna(text):
        return ""

    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply preprocessing
tqdm.pandas()
df['processed_text'] = df['Consumer complaint narrative'].progress_apply(preprocess_text)

# Drop rows with empty processed text
df = df[df['processed_text'] != '']
print(f"Dataset shape after preprocessing: {df.shape}")

# Split the data
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['category']
)

print(f"Training set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")

In [ ]:
# Create a simple TF-IDF + Logistic Regression model as our lightweight baseline
def create_lightweight_model():
    # Create TF-IDF features
    vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

    # Fit on training data
    X_train = vectorizer.fit_transform(train_df['processed_text'])
    X_test = vectorizer.transform(test_df['processed_text'])

    y_train = train_df['category']
    y_test = test_df['category']

    # Train a logistic regression model
    model = LogisticRegression(max_iter=1000, n_jobs=-1)

    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Make predictions
    start_time = time.time()
    y_pred = model.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    # Print results
    print(f"Lightweight Model (TF-IDF + Logistic Regression)")
    print(f"Training time: {training_time:.2f} seconds")
    print(f"Prediction time: {prediction_time:.2f} seconds")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Mean Squared Error: {mse:.4f}")

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Credit', 'Debt', 'Loan', 'Mortgage'],
                yticklabels=['Credit', 'Debt', 'Loan', 'Mortgage'])
    plt.title('Confusion Matrix - Lightweight Model')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred,
                               target_names=['Credit', 'Debt', 'Loan', 'Mortgage']))

    return {
        'model': model,
        'vectorizer': vectorizer,
        'accuracy': accuracy,
        'mse': mse,
        'confusion_matrix': cm,
        'training_time': training_time,
        'prediction_time': prediction_time
    }

# Run the lightweight model
lightweight_results = create_lightweight_model()

In [ ]:
# Define a PyTorch Dataset for our text data
class ComplaintDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels.iloc[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Function to train and evaluate a transformer model
def train_transformer_model(model_name):
    print(f"\nTraining {model_name} model...")

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=4  # We have 4 categories
    )

    # Create datasets
    train_dataset = ComplaintDataset(
        train_df['processed_text'],
        train_df['category'],
        tokenizer
    )
    test_dataset = ComplaintDataset(
        test_df['processed_text'],
        test_df['category'],
        tokenizer
    )

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=100,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        report_to="none",  # Disable wandb/tensorboard reporting
    )

    # Define metrics computation function
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        accuracy = accuracy_score(labels, predictions)
        mse = mean_squared_error(labels, predictions)
        return {
            'accuracy': accuracy,
            'mse': mse
        }

    # Create Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Train the model
    start_time = time.time()
    trainer.train()
    training_time = time.time() - start_time

    # Evaluate the model
    start_time = time.time()
    predictions = trainer.predict(test_dataset)
    prediction_time = time.time() - start_time

    # Get predicted labels
    y_pred = np.argmax(predictions.predictions, axis=1)
    y_true = test_df['category'].values

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    # Print results
    print(f"{model_name} Model")
    print(f"Training time: {training_time:.2f} seconds")
    print(f"Prediction time: {prediction_time:.2f} seconds")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Mean Squared Error: {mse:.4f}")

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Credit', 'Debt', 'Loan', 'Mortgage'],
                yticklabels=['Credit', 'Debt', 'Loan', 'Mortgage'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred,
                               target_names=['Credit', 'Debt', 'Loan', 'Mortgage']))

    return {
        'model': model,
        'tokenizer': tokenizer,
        'accuracy': accuracy,
        'mse': mse,
        'confusion_matrix': cm,
        'training_time': training_time,
        'prediction_time': prediction_time
    }

# Train a lightweight transformer model
distilbert_results = train_transformer_model('distilbert-base-uncased')

In [ ]:
# Create a comparison table
models = {
    'TF-IDF + Logistic Regression': lightweight_results,
    'DistilBERT': distilbert_results
}

comparison_df = pd.DataFrame({
    'Model': list(models.keys()),
    'Accuracy': [m['accuracy'] for m in models.values()],
    'MSE': [m['mse'] for m in models.values()],
    'Training Time (s)': [m['training_time'] for m in models.values()],
    'Prediction Time (s)': [m['prediction_time'] for m in models.values()]
})

# Display the comparison table
print("Model Performance Comparison:")
display(comparison_df)

# Visualize the comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy comparison
sns.barplot(x='Model', y='Accuracy', data=comparison_df, ax=axes[0, 0])
axes[0, 0].set_title('Model Accuracy Comparison')
axes[0, 0].set_ylim(0, 1)
for i, acc in enumerate(comparison_df['Accuracy']):
    axes[0, 0].text(i, acc + 0.01, f'{acc:.4f}', ha='center')

# MSE comparison
sns.barplot(x='Model', y='MSE', data=comparison_df, ax=axes[0, 1])
axes[0, 1].set_title('Model MSE Comparison')
for i, mse in enumerate(comparison_df['MSE']):
    axes[0, 1].text(i, mse + 0.01, f'{mse:.4f}', ha='center')

# Training time comparison
sns.barplot(x='Model', y='Training Time (s)', data=comparison_df, ax=axes[1, 0])
axes[1, 0].set_title('Model Training Time Comparison')
for i, t in enumerate(comparison_df['Training Time (s)']):
    axes[1, 0].text(i, t + 5, f'{t:.2f}s', ha='center')

# Prediction time comparison
sns.barplot(x='Model', y='Prediction Time (s)', data=comparison_df, ax=axes[1, 1])
axes[1, 1].set_title('Model Prediction Time Comparison')
for i, t in enumerate(comparison_df['Prediction Time (s)']):
    axes[1, 1].text(i, t + 0.5, f'{t:.2f}s', ha='center')

plt.tight_layout()
plt.show()

In [ ]:
# Select the best model based on accuracy
best_model_name = comparison_df.loc[comparison_df['Accuracy'].idxmax(), 'Model']
print(f"Best performing model: {best_model_name}")

# Function to make predictions on new data
def predict_complaint(text, model_name):
    # Preprocess the text
    processed_text = preprocess_text(text)

    if model_name == 'TF-IDF + Logistic Regression':
        # Use the lightweight model
        vectorized_text = lightweight_results['vectorizer'].transform([processed_text])
        prediction = lightweight_results['model'].predict(vectorized_text)[0]
        probabilities = lightweight_results['model'].predict_proba(vectorized_text)[0]
    else:
        # Use the transformer model
        encoding = distilbert_results['tokenizer'](
            processed_text,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_tensors='pt'
        )

        with torch.no_grad():
            outputs = distilbert_results['model'](**encoding)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=-1).cpu().numpy()[0]
            prediction = np.argmax(probabilities)

    # Map prediction to category name
    category_names = ['Credit reporting', 'Debt collection', 'Consumer Loan', 'Mortgage']
    category_name = category_names[prediction]

    return {
        'prediction': prediction,
        'category_name': category_name,
        'probabilities': {
            category_names[i]: float(probabilities[i])
            for i in range(len(category_names))
        }
    }

# Test the prediction function with sample complaints
sample_complaints = [
    "I am writing to dispute incorrect information on my credit report. The report shows a late payment that was made on time.",
    "A debt collector has been calling me multiple times a day regarding a debt that I don't believe is mine.",
    "I took out a personal loan last year and the interest rate was much higher than what was initially advertised.",
    "My mortgage company failed to apply my payments correctly and is now claiming I'm behind on payments."
]

for i, complaint in enumerate(sample_complaints):
    result = predict_complaint(complaint, best_model_name)
    print(f"\nSample Complaint {i+1}:")
    print(f"Text: {complaint}")
    print(f"Predicted Category: {result['category_name']} (ID: {result['prediction']})")
    print("Probabilities:")
    for category, prob in result['probabilities'].items():
        print(f"  {category}: {prob:.4f}")

In [ ]:
# Save the best model
import pickle

if best_model_name == 'TF-IDF + Logistic Regression':
    # Save the lightweight model
    model_path = '/content/drive/MyDrive/complaint_classifier_lightweight.pkl'
    with open(model_path, 'wb') as f:
        pickle.dump({
            'model': lightweight_results['model'],
            'vectorizer': lightweight_results['vectorizer']
        }, f)
    print(f"Lightweight model saved to {model_path}")
else:
    # Save the transformer model
    model_path = '/content/drive/MyDrive/complaint_classifier_distilbert'
    distilbert_results['model'].save_pretrained(model_path)
    distilbert_results['tokenizer'].save_pretrained(model_path)
    print(f"Transformer model saved to {model_path}")